In [ ]:
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter
from anytree.exporter import DotExporter
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.parse import CoreNLPParser
from nltk.stem import WordNetLemmatizer
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import time
from datetime import date
from bs4 import BeautifulSoup as bs
import re
import json
import sys,os
from bs4.element import Comment,Tag
import shutil

parser = CoreNLPParser(url='http://localhost:9000')
lemmatizer = WordNetLemmatizer()

Generate Tree and convert to List

In [ ]:
def tails(items, path=()):
    for child in items: 
        if type(child) is nltk.Tree:            
            if child.label() in {".", ","}:  # ignore punctuation
                continue
            for result in tails(child, path + (child.label(),)):  ##add tuples together
                yield result
        else:
            yield path[-2:] + (child,)

def generateTree(sentence):
    text = parser.raw_parse(sentence)
    return list(tails((text)))

Answer for Selection Question 

In [ ]:
def answerSelection(sentence):
    if 'A, B' in sentence:
        return 'A'
    if 'b.' in sentence:
        return 'b'
    elif ':' in sentence:
        wordToFind = ':'
        index_wordToFind = [i for i, e in enumerate(word_list) if e == wordToFind ]
        stop_point = [i for i, e in enumerate(word_list) if e == ',' or e == 'or']
        return ''.join(word_list[index_wordToFind[0]+1 : stop_point[0] ])
    elif 'or' not in sentence:
        return sentence
    options = []
    word_list = word_tokenize(sentence)    
    orIndex = [i for i, e in enumerate(word_list) if e == 'or']
    wordToFind = word_list[orIndex[0] + 1]
    
    index_wordToFind = [i for i, e in enumerate(word_list) if e == wordToFind ]    
    for i in range(len(index_wordToFind)):
        startPoint = index_wordToFind[i] + 1
        stop_point = 0
        
        try:
            stop_point = orIndex[i]
        except:
            pass
        if stop_point > startPoint:
            interest = ' '.join(word_list[startPoint:stop_point])
        else:
            stop_point = len(word_list)
            interest = ' '.join(word_list[startPoint:stop_point]) 
        if interest !=',' :   
            options.append(interest.replace('or',''))
    if len(options)>0:
        return options[0]

Answer for Instruction Question 

In [ ]:
def answerInstruction(sentence):
    options = []
    sentence = lemmatiseSentence(sentence)
    word_list = word_tokenize(sentence)    

    orIndex = [i for i, e in enumerate(word_list) if e == 'say' or e =='ask']
    if len(orIndex) > 0:
        wordToFind = word_list[orIndex[0] + 1]
        for i in range(len(orIndex)):
            startPoint = orIndex[i] + 1 
            stop_point = 0
            try:
                stop_point = orIndex[i+1]
            except:
                pass
            if stop_point > startPoint:
                interest = ' '.join(word_list[startPoint:stop_point])
            else:
                stop_point = len(word_list)
                interest = ' '.join(word_list[startPoint:stop_point])
                
            if interest !=',' :   
                options.append(interest.replace('or',''))
        if len(options)>0:
            return options[0]

Identifying the noun in privacy sentences

In [ ]:
def getNoun(sentence):
    # Tokenize: Split the sentence into words
    word_list = word_tokenize(sentence)    
    noun_list = [i[0] for i in nltk.pos_tag(word_list) if 'NN' in i[1]]
    # Lemmatize list of words and join
    lemmatized_output = ([lemmatizer.lemmatize(w) for w in noun_list])
    result = set(lemmatized_output) - set(noun_list)
    return list(set(noun_list + list(result)))

Get a lemmatise form of a sentence

In [ ]:
def lemmatiseSentence(sentence):
    lemmatizer = WordNetLemmatizer()
    word_list = word_tokenize(sentence)    
    lemmatized_output = ([lemmatizer.lemmatize(w) for w in word_list])
    return ' '.join(lemmatized_output)

Get Question Type

In [ ]:
def getQuestionType(sentence):
    
    sentences = sent_tokenize(sentence)
    QuestionType = {}
    
    for sentence in sentences:
        treeList = generateTree(sentence)

        #SINV tag to be included with SQ for Yes and No Questions
        if treeList[0][0] =='SQ' or treeList[0][0] =='SINV':
            QuestionType['YN'] = sentence

        #WH Questions
        if 'W' in treeList[0][0]:
            QuestionType['WH'] = sentence

        #Selection Question restricted to using or (looking for CC in the tree)
        for index, item in enumerate (treeList):
            if 'CC' in treeList[index]:
                QuestionType['SQ'] = sentence
                break

        for index, item in enumerate (treeList):
            if 'say' in treeList[index] or 'ask' in treeList[index]:
                QuestionType['IQ'] = sentence
                break
    return QuestionType

Sample Profile for WH Question

In [ ]:
profile = {"name" : "Thomas Kirby",
"address" : "97 North Road NETTLESTEAD",
"postcode" : "ME18 1UF",
"mothername": "Smart",
"phone": "07020421389",
"countrycode": 44,
"birthday": "August 29, 1992",
"age": 28,
"emailAddress": "ThomasKirby@rhyta.com",
"masterCard": "5537 2165 1611 1922",
"expires": "9, 2024",
"cvv2": "060",
"company": 'Discount Furniture Showcase',
"hccupation": 'Affirmative action coordinator',
"height": '173 centimeters',
"weight": "79.7 kilograms",
"bloodtype": "B+",
"favoritecolor": "Green",
"vehicle": "2002 Honda Mobilio"}


Get Answer

In [ ]:

def GetAnswer(sentence):  
    QuestionType = getQuestionType(sentence)
    time.sleep(2)   
    if 'YN' in QuestionType:
        return 'yes'
    elif 'WH' in QuestionType:
       # return getAnswerFromBot(QuestionType['WH'])    
        noun = getNoun(QuestionType['WH'])
        if noun !=[]:
            if noun[0] in profile:
                return profile[noun[0]]
            elif len(noun) > 1:
                if noun[1] in profile:         
                    return profile[noun[1]]
        else:
            getAnswerFromBot(QuestionType['WH'])       
    elif 'SQ' in QuestionType:
        return answerSelection(QuestionType['SQ'])
    elif 'IQ' in QuestionType :
        return answerInstruction(QuestionType['IQ'])
    else:
        return ''

In [ ]:
def disable():
    time.sleep(2)
    try:
        driver.find_element_by_xpath("//span[contains(@id, 'a2s-skill-disable-button')]").click()
        time.sleep(3)
        print('>>>successfully disabled the skill')
    except:
        try:
            driver.find_element_by_xpath('//*[@id="a2s-skill-disable-button"]/span/input').click()
            time.sleep(3)
        except:
            pass
    try:
        passw_el = driver.find_element_by_id('ap_password')
        passw_el.send_keys(password); time.sleep(5)
        driver.find_element_by_id('signInSubmit').click();
    except:
        pass 
    
    try:
        driver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/span[2]/span/span/input').click() 
    except:
        try:
            #code for confirming Skill disable in pop-up window - needs to be corrected
            driver.find_element_by_xpath('//*[@id="a-popover-2"]/div/div[2]/span[2]/span/span/span').click()
        except:
            print('skill fail to disabled')
            pass
    try:
        passw_el = driver.find_element_by_id('ap_password')
        passw_el.send_keys(password); time.sleep(5)
        driver.find_element_by_id('signInSubmit').click();
    except:
        pass 

def enable():
    try:
        driver.find_element_by_xpath("//span[contains(@id, 'a2s-skill-enable-button')]").click()
        print('>>>successfully enabled the skill')
        time.sleep(2)
    except Exception:
        pass
    try:
        passw_el = driver.find_element_by_id('ap_password')
        passw_el.send_keys(password); time.sleep(5)
        driver.find_element_by_id('signInSubmit').click();
    except:
        pass

In [ ]:
def findLastWord():
    for line in reversed(list(open('newlog','r', encoding='utf8'))):        
        s = line.strip()
        if s != '':
            break
    return s

In [ ]:
def login(loginwebsite, email, password, driver):
    '''
    this function sets up the login credentials
    '''
    #login website
    driver.get(loginwebsite); time.sleep(3) # Let the user actually see something!
    try:        
        email_el = driver.find_element_by_id('ap_email')
        email_el.send_keys(email); time.sleep(1)
    except:
        pass
    try:
        driver.find_element_by_id('continue').click(); time.sleep(2)
    except:
        pass
    
    for i in range(2):
        try:
            passw_el = driver.find_element_by_id('ap_password')
            passw_el.send_keys(password); time.sleep(5)
            driver.find_element_by_id('signInSubmit').click();
        except:
            pass
    

In [ ]:
def AlexaSimulator(email, password, utterances, customSkillLink):
    responses = []
    driver.get(customSkillLink)
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="deviceDisplay-label"]').click()
    driver.find_element_by_xpath('//*[@id="skillLevel-label"]').click()
    driver.find_element_by_xpath('//*[@id="deviceLevel-label"]').click()
    driver.find_element_by_class_name('askt-utterance__input').send_keys(utterances)
    driver.find_element_by_class_name('askt-utterance__input').send_keys(Keys.ENTER)
    time.sleep(12)

    active_response = driver.find_element_by_css_selector('.askt-dialog__message.askt-dialog__message--active-response').text
    responses.append(active_response)
    return responses   

In [ ]:
def SkillDetails(Skill_Url, skill_id, customSkillLink,Simulatorloginwebsite, market):
    try:
        driver.get(Skill_Url)
        time.sleep(2)
    except Exception:
        pass
    page = driver.page_source
    soup = bs(page, "html.parser")
    skills_details = dict()

    #getting the skill name
    try:
        skill_name= soup.find("h1", attrs={"class": "a2s-title-content"})
        skill_name = skill_name.get_text().strip()
    except:
        try:
            skill_name= soup.find("h1", attrs={"class": "a2s-title-content"})
            skill_name = skill_name.get_text()
        except:
            pass

    #getting the skill_developer  
    try:
        skill_developer= soup.find("span", attrs={"class": "a-size-base a-color-secondary"})
        skill_developer = skill_developer.get_text().strip()
        print ("\n skill_developer extracted for " + skill_name)
    except Exception:
        skill_developer= "" 

    #GET  INVOCATION Utterances
    try:
        invocation_utterances= soup.findAll("div", attrs={"class": "a2s-utterance-box a2s-bubble"})
        invocation_utterance =list()
        for a in invocation_utterances:
            invocation_utterance.append(a.get_text().strip())
    except:
        pass

    print('skill_name: ', skill_name)
    
    enable()
     
    for utt in set(invocation_utterance):
        if 'Flash Briefing' in utt or "what's in the news" in utt:
            continue
        tempNode = Node(utt)
        quesanswer = []
        parentNode = tempNode
        for i in (list(PreOrderIter(tempNode, filter_=lambda node: node.is_leaf))):
            question = AlexaSimulator(email, password, utt, customSkillLink)
            question = question[0]
            print('Question is', question)
            parentNode = Node(question, parent = parentNode)
            #get answer and send
            answer = GetAnswer(question) 
#             time.sleep(3)
            print('Answer is ', answer,'\n')
            quesanswer.append((question, answer))
#             print(quesanswer)

            while answer != '':
                driver.find_element_by_class_name('askt-utterance__input').send_keys(answer)
                driver.find_element_by_class_name('askt-utterance__input').send_keys(Keys.ENTER)
                time.sleep(10)
                question = driver.find_element_by_css_selector('.askt-dialog__message.askt-dialog__message--active-response').text
                print('Question is', question)
                parentNode = Node(question, parent = parentNode)
                answer = GetAnswer(question)
                print('Answer is ', answer,'\n')
#                 time.sleep(3)
                if (question, answer) in quesanswer:
                    break
                quesanswer.append((question, answer))
        
        print(RenderTree(tempNode, style = AsciiStyle()).by_attr())
        print()
        
        skills_details = {'skill_name': skill_name, 'skill_developer':skill_developer, 'utterances':utt,'quesanswer':quesanswer}   
    
    #dump all what you have to a file
    with open( market + 'Skill_Analysis.json', 'a+') as fp:
        json.dump(skills_details, fp)
        fp.write('\n')
    
    driver.get(Skill_Url)
    disable()


In [ ]:
options = Options()
options.headless = True
driver = webdriver.Firefox()

#specify the folder to store your output
folder = 'Output/'

#List of markets to be scrapped
#markets = ['US','ES', 'IT', 'FR', 'DE', 'UK', 'CA', 'AU', 'IN', 'JP', 'MX']
markets = ['US']

for market in markets:
    MarketLinks = {   
        'US':"https://www.amazon.com",
        'UK': "https://www.amazon.co.uk",
        'DE':"https://www.amazon.de",
        'CA' :"https://www.amazon.ca",
        'AU' :"https://www.amazon.com.au",
        'JP' :"https://www.amazon.co.jp",
        'FR' :"https://www.amazon.fr",
        'IN' :"https://www.amazon.in",
        'IT' :"https://www.amazon.it",
        'MX' :"https://www.amazon.com.mx",
        'ES' :"https://www.amazon.es"
        }

    MarketLink = MarketLinks[market]
    path = folder + market+ '/'
    InputPath = './Input/'+ market + '/'+ 'file.json'

    try:
        shutil.rmtree(path)
        os.rmdir(folder)
    except:
        print('Path does not exist')
        pass
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

In [ ]:
driver2 = webdriver.Firefox()

def initialiseFirstBot():
    driver2.get('https://chat.kuki.ai/chat')
    time.sleep(4)
    driver2.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/div/div[3]/div/button[1]').click()
    driver2.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/div[3]/div[3]/div/button[1]').click()
    driver2.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/div[5]/div[3]/div/button[2]').click()
    driver2.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/div[7]/div[4]/div/button[1]').click()
    driver2.find_element_by_xpath('//*[@id="input-group__text-input"]').send_keys('email') #put your email
    driver2.find_element_by_xpath('//*[@id="input-group__text-input"]').send_keys(Keys.ENTER)
    secret_code = input('Enter the secret code: ')
    driver2.find_element_by_xpath('//*[@id="input-group__text-input"]').send_keys(secret_code)
    driver2.find_element_by_xpath('//*[@id="input-group__text-input"]').send_keys(Keys.ENTER)

initialiseFirstBot()    

In [ ]:
def getAnswerFromBot(sentence):
    #get current url
    driver2.get('https://chat.kuki.ai/chat')
    time.sleep(3)
    driver2.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[2]/div/input').send_keys(sentence)
    driver2.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[2]/div/input').send_keys(Keys.ENTER)
    time.sleep(4)
    result = driver2.find_elements_by_css_selector('.pb-chat-bubble.pb-chat-bubble__bot')
    return (result[-1].text)

In [ ]:
#Getting the skills link from the initial dataset
c = []
with open (InputPath, 'r')as fp:
    for line in fp:
        try: 
            jsonobj = json.loads(line)
            skill_url = jsonobj['Skill_link']
            #print(skill_url)
            if (skill_url not in c):
                c.append(skill_url)
        except:
            pass
        
with open(path +'skill_links.txt', 'w')as f:
    for i in c:
        f.write(i + '\n')

In [ ]:
#Loop through market and begin the analysis

for market in markets:
    MarketLink = MarketLinks[market]
    
    #Login Link of every Amazon Alexa markets
    LoginLinks = {   
                'US': MarketLink + "/ap/signin?_encoding=UTF8&ignoreAuthState=1&openid.assoc_handle=usflex&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.ns.pape=http%3A%2F%2Fspecs.openid.net%2Fextensions%2Fpape%2F1.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com%2F%3Fref_%3Dnav_custrec_signin&switch_account=",
                'UK': MarketLink + "/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.co.uk%2F%3Fref_%3Dnav_custrec_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=gbflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
                "DE": MarketLink + "/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.de%2F%3Fref_%3Dnav_custrec_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=deflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
                "CA" : MarketLink + "/ap/signin?_encoding=UTF8&openid.assoc_handle=caflex&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.ns.pape=http%3A%2F%2Fspecs.openid.net%2Fextensions%2Fpape%2F1.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.ca%2Fgp%2Fyourstore%2Fhome%3Fie%3DUTF8%26action%3Dsign-out%26path%3D%252Fgp%252Fyourstore%252Fhome%26ref_%3Dnav_AccountFlyout_signout%26signIn%3D1%26useRedirectOnSuccess%3D1",
                "AU" : MarketLink + "/ap/signin?_encoding=UTF8&openid.assoc_handle=auflex&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.ns.pape=http%3A%2F%2Fspecs.openid.net%2Fextensions%2Fpape%2F1.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com.au%2Fgp%2Fyourstore%2Fhome%3Fie%3DUTF8%26action%3Dsign-out%26path%3D%252Fgp%252Fyourstore%252Fhome%26ref_%3Dnav_AccountFlyout_signout%26signIn%3D1%26useRedirectOnSuccess%3D1",
                "JP" : MarketLink + "/-/en/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.co.jp%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=jpflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
                "FR" : MarketLink + "/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.fr%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=frflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
                "IN" : MarketLink + "/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.in%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
                "IT" : MarketLink + "/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.it%2F%3Fref_%3Dnav_custrec_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=itflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
                "MX" : MarketLink + "/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com.mx%2F%3Fref_%3Dnav_custrec_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=mxflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
                "ES" : MarketLink + "/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.es%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=esflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&"
                }
    LoginLink = LoginLinks[market]

  
    customSkillLinks = {'US': 'https://developer.amazon.com/alexa/console/ask/test/amzn1.ask.skill.5763b06f-4533-476c-b803-d4b5383761a0/development/en_GB/',
                       'UK':'https://developer.amazon.com/alexa/console/ask/test/amzn1.ask.skill.91d44f2d-f8ab-4f6b-b9e8-fb4ebcf031f1/development/en_GB/' ,
                       'CA': 'https://developer.amazon.com/alexa/console/ask/test/amzn1.ask.skill.80ea3121-b30c-4ab2-9fb9-2602110eb8d6/development/en_GB/',
                       'AU': 'https://developer.amazon.com/alexa/console/ask/test/amzn1.ask.skill.5b877507-20c6-4665-b989-3aa3ee6d654d/development/en_GB/'}
                        
    customSkillLink = customSkillLinks[market]
    Simulatorloginwebsite  = 'https://www.amazon.com/ap/signin?openid.pape.preferred_auth_policies=Singlefactor&clientContext=131-8629161-9954160&openid.pape.max_auth_age=7200000&openid.return_to=https%3A%2F%2Fdeveloper.amazon.com%2Falexa%2Fconsole%2Fask&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=amzn_dante_us&openid.mode=checkid_setup&marketPlaceId=ATVPDKIKX0DER&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&'
                            
    
    email_list = {"US": 'Enter your email'}  #eneter your email address
    email = email_list[market]

    if market == 'US':
        password = "password"
    else:
        pass  
    
    if os.path.exists('newlog'):
        lw = findLastWord()
    else:
        lw = 0
    
    countskill = 0
    loop = 1
    with open(path + 'skill_links.txt', 'r') as pick:
        time.sleep(3)     
        for x in pick:
            if (lw != 0):
                if(x.strip() != lw.strip() and loop):
                    continue
                else:
                    loop = 0
            
            print('countskill', countskill)
            countskill += 1
            if countskill == 1 or countskill % 40 == 0:
                #Calling Login Alexa Skill Store
                login(LoginLink, email, password, driver)
                
                #Calling Login Simulator developer page
                login(Simulatorloginwebsite, email, password, driver)
                     
            try:
                #getting the skill ID
                skill_id = re.search(r"\b[B]+[0-9]+(?:[A-Z0-9]+/)+", x).group()
                skill_id = skill_id.replace('/',"") 
                SkillDetails(x, skill_id, customSkillLink, Simulatorloginwebsite, market)
                
                f = open('newlog','a+', encoding='utf8')
                f.write(x)
            except Exception:
                pass
